In [5]:
import cv2
import numpy as np
from imageio import imread
from PIL import Image
import os

In [6]:
def readFlow(path):
        assert type(path) is str, "Input [{p}] is not a string".format(p=path)
        assert os.path.isfile(path), "Path [{p}] does not exist".format(p=path)
        assert path.split('.')[-1] == 'flo', "File extension [flo] required, [{f}] given".format(f=path.split('.')[-1])

        flo = open(path, 'rb')

        tag = np.fromfile(flo, np.float32, count=1)[0]
        assert 202021.25 == tag, "Wrong Tag [{t}]".format(t=tag)

        width = np.fromfile(flo, np.int32, count=1)[0]
        assert width > 0 and width < 100000, "Illegal width [{w}]".format(w=width)

        height = np.fromfile(flo, np.int32, count=1)[0]
        assert width > 0 and width < 100000, "Illegal height [{h}]".format(h=height)

        nbands = 2
        tmp = np.fromfile(flo, np.float32, count= nbands * width * height)
        flow = np.resize(tmp, (int(height), int(width), int(nbands)))
        flo.close()
        
        return flow

def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = cv2.normalize(v, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow[:,:,0], flow[:,:,1], cv2.INTER_LINEAR)
    #res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return res




In [ ]:
im1 = np.asarray(cv2.imread("i7.png"))
flow = readFlow("i78.flo")

im2w = warp_flow(im1, flow)

cv2.imwrite("i78_pwc.jpg", im2w)

In [9]:
flow = readFlow("./images/out21.flo")
print(flow.shape)


(436, 1024, 2)


In [7]:
im1 = np.asarray(cv2.imread("./images/second.png"))
flow = readFlow("./images/out21.flo")

im2w = warp_flow(im1, flow)

cv2.imwrite("./images/flow_pwc2.png", im2w)

True

In [ ]:
im1 = np.asarray(cv2.imread("i7.png", 0))
im2 = np.asarray(cv2.imread("i8.png",0))
flow = cv2.calcOpticalFlowFarneback(im1,im2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

im2ref = np.asarray(cv2.imread("i7.png"))
im2w = warp_flow(im2ref, flow)

cv2.imwrite("i78wW_1.jpg", im2w)

In [ ]:
tvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
im1 = np.asarray(cv2.imread("i7.png", 0))
im2 = np.asarray(cv2.imread("i8.png",0))
flow = tvl1.calc(im1,im2, None)

im2ref = np.asarray(cv2.imread("i7.png"))
im2w = warp_flow(im2ref, flow)

cv2.imwrite("i78wW_tvl1.jpg", im2w)

In [17]:
tvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
im1 = np.asarray(cv2.imread("./images/first.png", 0))
im2 = np.asarray(cv2.imread("./images/second.png",0))
flow = tvl1.calc(im1,im2, None)

im2ref = np.asarray(cv2.imread("./images/first.png"))
im2w = warp_flow(im2ref, flow)

cv2.imwrite("./images/flow_tvl1.jpg", im2w)

True